# Import

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

# Parse the results

In [ ]:
result = {}

families = set()
instances = set()
kValue = set()
ratio = set()
instanceToFamily = {}


with open('result.res') as f:
    lines = f.readlines()
    
    entry = {}
    for line in lines:
        if line[0] == '{':
            entry = {}
        elif line[0] == '}':
            families.add(entry['family'])
            instances.add(entry['instance']) 
            instanceToFamily[entry['instance']] = entry['family']
            
            if entry['method'] not in result:
                result[entry['method']] = {}
                        
            ptrMethod = result[entry['method']]
            
            if entry['method'] == 'tf' or entry['method'] == 'trivial':
                if entry['family'] not in ptrMethod:
                    ptrMethod[entry['family']] = {}
                ptrMethod[entry['family']][entry['instance']] = entry                
            elif entry['method'] == 'ktf':
                kValue.add(entry['k'])
                if entry['k'] not in ptrMethod:
                    ptrMethod[entry['k']] = {}
                if entry['family'] not in ptrMethod[entry['k']]:
                    ptrMethod[entry['k']][entry['family']] = {}
                ptrMethod[entry['k']][entry['family']][entry['instance']] = entry
            elif entry['method'] == 'rtf':
                if entry['k'] not in ptrMethod:
                    ptrMethod[entry['k']] = {}                
                if entry['cut'] not in ptrMethod[entry['k']]:
                    ptrMethod[entry['k']][entry['cut']] = {}
                if entry['family'] not in ptrMethod[entry['k']][entry['cut']]:
                    ptrMethod[entry['k']][entry['cut']][entry['family']] = {}
                ptrMethod[entry['k']][entry['cut']][entry['family']][entry['instance']] = entry
            else:
                ratio.add(entry['t'])
                
                if entry['k'] not in ptrMethod:
                    ptrMethod[entry['k']] = {}                
                if entry['cut'] not in ptrMethod[entry['k']]:
                    ptrMethod[entry['k']][entry['cut']] = {}                    
                if entry['t'] not in ptrMethod[entry['k']][entry['cut']]:
                    ptrMethod[entry['k']][entry['cut']][entry['t']] = {}
                if entry['family'] not in ptrMethod[entry['k']][entry['cut']][entry['t']]:
                    ptrMethod[entry['k']][entry['cut']][entry['t']][entry['family']] = {}                
                ptrMethod[entry['k']][entry['cut']][entry['t']][entry['family']][entry['instance']] = entry
        else:
            key = line.split(':')[0].replace(" ", "")
            value = line.split(':')[1].replace(" ", "")[:-1]
            if value[-1] == ',':
                value = value[:-1]            
            entry[key] = value            

## Some useful variables

In [ ]:
resMapNbSolved = {}

families = list(families)
families.sort()

ratio = list(ratio)
ratio.sort()

kValue = list(kValue)
kValue.sort()

TIMEOUT = 3700


mapNameMethod = {'trivial' : 'trivial', 'tf' : 'tf', 'ktf' : 'k-rob', 'rtf' : 'k-rec', 'ptf' : 'p-rob', 'dptf' : 'p-rob*'}

familyColor = {}
familyColor['1-rtf30-TF-instances'] = 'blue'
familyColor['2-cire100-TF-instances'] = 'darkviolet'
familyColor['3-cire150-TF-instances'] = 'indigo'
familyColor['4-scp1000-TF-instances'] = 'bisque'
familyColor['5-map-TF-instances'] = 'plum'
familyColor['r1-default'] = 'orange'
familyColor['r2-default'] = 'red'
familyColor['r3-default'] = 'green'
familyColor['r4-default'] = 'magenta'
familyColor['r5-default'] = 'yellow'
familyColor['r6-default'] = 'purple'
familyColor['r7-default'] = 'gray'

familyShortName = {}
familyShortName['1-rtf30-TF-instances'] = 'rtf30'
familyShortName['2-cire100-TF-instances'] = 'cire100'
familyShortName['3-cire150-TF-instances'] = 'cire150'
familyShortName['4-scp1000-TF-instances'] = 'scp1000'
familyShortName['5-map-TF-instances'] = 'map'
familyShortName['r1-default'] = 'r1'
familyShortName['r2-default'] = 'r2'
familyShortName['r3-default'] = 'r3'
familyShortName['r4-default'] = 'r4'
familyShortName['r5-default'] = 'r5'
familyShortName['r6-default'] = 'r6'
familyShortName['r7-default'] = 'r7'

familySymbol = {}
familySymbol['1-rtf30-TF-instances'] = 'v'
familySymbol['2-cire100-TF-instances'] = '>'
familySymbol['3-cire150-TF-instances'] = '<'
familySymbol['4-scp1000-TF-instances'] = '^'
familySymbol['5-map-TF-instances'] = '*'
familySymbol['r1-default'] = 'o'
familySymbol['r2-default'] = 'p'
familySymbol['r3-default'] = '+'
familySymbol['r4-default'] = 'd'
familySymbol['r5-default'] = 's'
familySymbol['r6-default'] = '1'
familySymbol['r7-default'] = '2'

# Cut evaluation

## No cut vs. 'classic' cut 

In [ ]:
mSize = 150

for method in ['ptf', 'dptf']:
    for t in ['90', '95', '99']:
        for k in kValue:  
            vk = int(k)            
            x = range(0,4000)
            y = x

            plt.rcParams["figure.figsize"] = (10,10)
            plt.plot(x,y)
            plt.xlabel(mapNameMethod[method], fontsize=15)
            plt.ylabel(mapNameMethod[method] + '(cut)', fontsize=15)
            plt.xscale('log')
            plt.yscale('log')

            for f in families:
                # get the points by family
                point0 = []
                point1 = []

                for i in instances:
                    if instanceToFamily[i] != f:
                        continue

                    entry0 = result[method][k]['0'][t][f][i]
                    entry1= result[method][k]['1'][t][f][i]                

                    if(entry0['status'] == 'solved' and entry0['optimal'] == 'True'):
                        point0.append(float(entry0['time']))
                    else: 
                        point0.append(TIMEOUT)

                    if(entry1['status'] == 'solved' and entry1['optimal'] == 'True'):
                        point1.append(float(entry1['time']))
                    else:
                        point1.append(TIMEOUT)

                plt.scatter(point0, point1, marker=familySymbol[f],s=mSize, c=familyColor[f],label=familyShortName[f])

            leg = plt.legend(loc="upper left", bbox_to_anchor=[0, 1],
                         ncol=2, shadow=True, title="k = " + k + ', t = ' + t, fancybox=True,fontsize=15)
            leg.get_title().set_color("red")            
            plt.show()


## 'classic' cut vs. cut+

In [ ]:
for method in ['ptf', 'dptf']:
    for t in ['90', '95', '99']:
        for k in kValue:  
            vk = int(k)
            x = range(0,4000)
            y = x

            plt.rcParams["figure.figsize"] = (10,10)
            plt.plot(x,y)
            plt.xlabel(mapNameMethod[method] + '(cut+)', fontsize=15)
            plt.ylabel(mapNameMethod[method] + '(cut)', fontsize=15)
            plt.xscale('log')
            plt.yscale('log')

            for f in families:
                # get the points by family
                point0 = []
                point1 = []

                for i in instances:
                    if instanceToFamily[i] != f:
                        continue

                    entry0 = result[method][k]['2'][t][f][i]
                    entry1= result[method][k]['1'][t][f][i]                

                    if(entry0['status'] == 'solved' and entry0['optimal'] == 'True'):
                        point0.append(float(entry0['time']))
                    else: 
                        point0.append(TIMEOUT)                      

                    if(entry1['status'] == 'solved' and entry1['optimal'] == 'True'):
                        point1.append(float(entry1['time']))
                    else:
                        point1.append(TIMEOUT)

                plt.scatter(point0, point1, marker=familySymbol[f], s=mSize,c=familyColor[f],label=familyShortName[f])

            leg = plt.legend(loc="upper left", bbox_to_anchor=[0, 1],
                         ncol=2, shadow=True, title="k = " + k + ', t = ' + t, fancybox=True,fontsize=15)
            leg.get_title().set_color("red")


            plt.tight_layout()
            plt.show()


# Print table

In [ ]:
# compute for tf
res = {}
cpt = 0        
for family in families:
    intermediateCpt = 0

    for instance in result['tf'][family]:
        if result['tf'][family][instance]['status'] == 'solved':
            intermediateCpt += 1
    res[family] = intermediateCpt
    cpt += intermediateCpt
res['total'] = cpt
resMapNbSolved['tf'] = res

In [ ]:
# compute for ktf
resMapNbSolved['ktf'] = {}

for k in kValue:
    res = {}
    cpt = 0        
    for family in families:
        intermediateCpt = 0
        for instance in result['ktf'][k][family]:
            if result['ktf'][k][family][instance]['status'] == 'solved':
                intermediateCpt += 1
        res[family] = intermediateCpt
        cpt += intermediateCpt
    res['total'] = cpt
    resMapNbSolved['ktf'][k] = res    

In [ ]:
# compute for robTF
resMapNbSolved['rtf'] = {}

for k in kValue:
    resMapNbSolved['rtf'][k] = {}
    for cut in ['0', '1']:
        res = {}
        cpt = 0        
        for family in families:
            intermediateCpt = 0
            for instance in result['rtf'][k][cut][family]:
                if result['rtf'][k][cut][family][instance]['status'] == 'solved':
                    intermediateCpt += 1
            res[family] = intermediateCpt
            cpt += intermediateCpt
        res['total'] = cpt
        resMapNbSolved['rtf'][k][cut] = res

In [ ]:
# compute for ptf and dptf
resMapNbSolved['ptf'] = {}
resMapNbSolved['dptf'] = {}

for method in ['ptf', 'dptf']:
    for k in kValue:
        resMapNbSolved[method][k] = {}
        for cut in ['0', '1', '2']:
            resMapNbSolved[method][k][cut] = {}
            for r in ratio:
                res = {}
                cpt = 0        
                for family in families:
                    intermediateCpt = 0
                    
                    if cut not in result[method][k]:
                        print("problem", method, k, cut)
                        continue
                    for instance in result[method][k][cut][r][family]:
                        if result[method][k][cut][r][family][instance]['status'] == 'solved':
                            intermediateCpt += 1
                    res[family] = intermediateCpt
                    cpt += intermediateCpt
                res['total'] = cpt
                resMapNbSolved[method][k][cut][r] = res                



In [ ]:
from IPython.display import HTML, display

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4></td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

    
table = [['Family', 'Total', "k", "TF", "RobTF", "RecTF", "PR-TF(90)", "PR-TF(95)", "PR-TF(99)", "dPR-TF(90)", "dPR-TF(95)", "dPR-TF(99)"]]
for family in families:
    shortFamilyName = familyShortName[family]
    
    kValue.sort()
    for k in kValue:
        line = [shortFamilyName, len(result['tf'][family]), k]
        
        # tf result                
        line.append(resMapNbSolved['tf'][family])
        
        # ktf result        
        line.append(resMapNbSolved['ktf'][k][family])
        
        # rtf result
        line.append(str(resMapNbSolved['rtf'][k]['1'][family]))
        
        # ptf result
        for t in ['90', '95', '99']:
            res = ""
            res += str(resMapNbSolved['ptf'][k]['0'][t][family]) + " - "
            res += str(resMapNbSolved['ptf'][k]['1'][t][family]) + " - "
            res += str(resMapNbSolved['ptf'][k]['2'][t][family])
            line.append(res)
        
        # dptf result
        for t in ['90', '95', '99']:
            res = ""
            res += str(resMapNbSolved['dptf'][k]['0'][t][family]) + " - "
            res += str(resMapNbSolved['dptf'][k]['1'][t][family]) + " - "
            res += str(resMapNbSolved['dptf'][k]['2'][t][family])
            line.append(res)      
        
        table.append(line)
    
    
display_table(table)

# Cactus plots

In [ ]:
# get the tf points
listTime = []
for f in result['tf']:
    for i in result['tf'][f]:
        entry = result['tf'][f][i]
        if(entry['status'] == 'solved'):
            listTime.append(float(entry['time']))
listTime.sort()

In [ ]:
fig, ax = plt.subplots(2, 2,figsize=(12,10))

lineSize=3

rangeX = range(len(instances))
for k in kValue:  
    vk = int(k)
    p = ax[(vk-1)//2, (vk-1)%2]    
    p.plot(range(len(listTime)), listTime, '-',markersize=10, linewidth=lineSize, color='red', label=mapNameMethod['tf'])

    
    # print ktf
    lTmp = []
    for f in result['ktf'][k]:
        for i in result['ktf'][k][f]:
            entry = result['ktf'][k][f][i]
            if(entry['status'] == 'solved'):
                lTmp.append(float(entry['time']))
    lTmp.sort()
    p.plot(range(len(lTmp)), lTmp, '-', linewidth=lineSize,markersize=10, color='green', label=mapNameMethod['ktf'])
    
    # print rtf
    lTmp = []
    for f in result['rtf'][k]['1']:        
        for i in result['rtf'][k]['1'][f]:
            entry = result['rtf'][k]['1'][f][i]
            if(entry['status'] == 'solved'):
                lTmp.append(float(entry['time']))
    lTmp.sort()
    p.plot(range(len(lTmp)), lTmp, '-', linewidth=lineSize,markersize=10, color='blue', label=mapNameMethod['rtf'])

    # print ptf
    colorPtf = {'90' : 'indigo', '95': 'darkviolet', '99' : 'plum'}
    for t in ['90', '95', '99']:
        lTmp = []
        for f in result['ptf'][k]['2'][t]:
            for i in result['ptf'][k]['2'][t][f]:
                entry = result['ptf'][k]['2'][t][f][i]
                if(entry['status'] == 'solved'):
                    lTmp.append(float(entry['time']))
        lTmp.sort()
        name = '<' + k + ', '+ t + '>' + mapNameMethod['ptf']
        p.plot(range(len(lTmp)), lTmp, '-', linewidth=lineSize,markersize=10, color=colorPtf[t], label=name)
        
    # print dptf
    colorDtf = {'90' : 'darkorange', '95': 'tan', '99' : 'bisque'}
    for t in ['90', '95', '99']:
        lTmp = []
        for f in result['dptf'][k]['2'][t]:
            for i in result['dptf'][k]['2'][t][f]:
                entry = result['dptf'][k]['2'][t][f][i]
                if(entry['status'] == 'solved'):
                    lTmp.append(float(entry['time']))
        lTmp.sort()
        name = '<' + k + ', '+ t + '>' + mapNameMethod['dptf']
        p.plot(range(len(lTmp)), lTmp, '-', linewidth=lineSize,markersize=10, color=colorDtf[t], label=name)

    p.set_xlabel("Number of solved instances (k = " + k + ")", fontsize=15)
    p.set_ylabel("Time (in seconds)", fontsize=15)
    p.set_yscale('log')
        

handles_, labels_ = ax[0,0].get_legend_handles_labels()
fig.legend(handles=handles_, labels=labels_,loc='upper center', bbox_to_anchor=(0.5, 1.12),
          ncol=3, fancybox=True, shadow=True,fontsize=15)

plt.tight_layout()
plt.show()

# Scatter plots

## Comparison between p-rob and p-rob*

In [ ]:
for t in ['90', '95', '99']:
    for k in kValue:  
        vk = int(k)
        x = range(0,4000)
        y = x

        plt.rcParams["figure.figsize"] = (10,10)
        plt.plot(x,y)
        plt.xlabel(mapNameMethod['dptf'], fontsize=15)
        plt.ylabel(mapNameMethod['ptf'], fontsize=15)
        plt.xscale('log')
        plt.yscale('log')

        for f in families:
            # get the points by family
            pointDPtf = []
            pointPtf = []

            for i in instances:
                if instanceToFamily[i] != f:
                    continue
                    
                if i not in result['ptf'][k]['2'][t][f]:
                    print(k, t, f, i)
                entryDptf = result['dptf'][k]['2'][t][f][i]
                entryPtf = result['ptf'][k]['2'][t][f][i]

                if(entryDptf['status'] == 'solved' and entryDptf['optimal'] == 'True'):
                    pointDPtf.append(float(entryDptf['time']))
                else: 
                    pointDPtf.append(TIMEOUT)

                if(entryPtf['status'] == 'solved' and entryPtf['optimal'] == 'True'):
                    pointPtf.append(float(entryPtf['time']))
                else:
                    pointPtf.append(TIMEOUT)

            plt.scatter(pointDPtf, pointPtf, marker=familySymbol[f],s=mSize, c=familyColor[f],label=familyShortName[f])

        leg = plt.legend(loc="upper left", bbox_to_anchor=[0, 1],
                     ncol=2, shadow=True, title="k = " + k + ', t = ' + t, fancybox=True, fontsize=15)
        leg.get_title().set_color("red")    
        plt.show()


## Compare the cost (the baseline is k-rob for ptf).

In [ ]:
for t in ['90', '95', '99']:
    for k in kValue:  
        vk = int(k)   
        
        plt.rcParams["figure.figsize"] = (10,10)
        plt.xlabel(mapNameMethod['dptf'], fontsize=15)
        plt.ylabel(mapNameMethod['ptf'], fontsize=15)
        
        maxVal = 0
        minVal = 10000000
        
        for f in families:
            # get the points by family
            pointDPtf = []
            pointPtf = []
            
            for i in instances:
                if instanceToFamily[i] != f:
                    continue
                entryDptf = result['dptf'][k]['2'][t][f][i]
                entryPtf = result['ptf'][k]['2'][t][f][i]
                
                if entryDptf['cost'] != 'None' and entryPtf['cost'] != 'None' and abs(int(entryDptf['cost']) - int(entryPtf['cost'])) < 1:
                    continue
                
                if(entryDptf['cost'] != 'None'):
                    pointDPtf.append(int(entryDptf['cost']))
                else: 
                    continue

                if(entryPtf['cost'] != 'None'):
                    pointPtf.append(int(entryPtf['cost']))
                elif result['ktf'][k][f][i]['cost'] != 'None':
                    pointPtf.append(int(result['ktf'][k][f][i]['cost']))
                else:
                    pointDPtf.pop()
                    continue
                                
                
                if pointDPtf[-1] == pointPtf[-1]:
                    pointDPtf.pop()
                    pointPtf.pop()
                    continue
                minVal = min(minVal, min(pointDPtf[-1], pointPtf[-1]))
                maxVal = max(maxVal, max(pointDPtf[-1], pointPtf[-1]))
                    
            plt.scatter(pointDPtf, pointPtf, marker=familySymbol[f],s=mSize, c=familyColor[f],label=familyShortName[f])

        if maxVal > 1000:
            p.set_yscale('symlog')
            p.set_xscale('symlog')        
            
        x = range(minVal, maxVal)
        y = x
        plt.plot(x,y)
        
        leg = plt.legend(loc="lower right", bbox_to_anchor=[1, 0],
                     ncol=2, shadow=True, title="k = " + k + ', t = ' + t, fancybox=True,fontsize=15)
        leg.get_title().set_color("red")        
        plt.show()


# Cost evolution

In [ ]:
for t in ['90', '95', '99']:    
    for k in kValue:  
        for f in families:
            for i in instances:
                if instanceToFamily[i] != f:
                    continue
                    
                entry = result['dptf'][k]['2'][t][f][i]                
                if entry['optimal'] != 'True':
                    continue
                if float(entry['time']) < 100:
                    continue
                                
                indSol = entry['intermediate-solution'].split('*')
                if(len(indSol) < 3):
                    continue
                
                cost = float(entry['cost'])
                
                x = []
                y = []
                for s in indSol:
                    ss = s.replace("(", "").replace(")", "").split(',')
                    x.append(float(ss[1]))
                    y.append((cost / float(ss[0])))
                
                plt.rcParams["figure.figsize"] = (15,10)
                plt.title(i + " (" + k + ","+ t + ")",fontsize=15)
                plt.ylabel("Cost",fontsize=15)
                plt.xlabel("Time (in seconds)",fontsize=15)
                plt.plot(x, y,label="optimal cost / intermediate cost")
                plt.legend(loc="lower right",fontsize=15)
                plt.show()                

# Analyze

We report for each triple (method, k, t) and for each family the average cost, the average cost after rescue and the average ratio of skills covered if the worst cast when k agent are defective.

In [ ]:
analyze = {}

with open('analyze.res') as f:
    lines = f.readlines()
    
    entry = {}
    for line in lines:
        if line[0] == '{':
            entry = {}
        elif line[0] == '}':                                    
            if entry['method'] not in analyze:
                analyze[entry['method']] = {}                 
            ptrMethod = analyze[entry['method']]
            
            if entry['k'] not in ptrMethod:
                ptrMethod[entry['k']] = {}
            ptrMethod = ptrMethod[entry['k']]
            
            if entry['t'] not in ptrMethod:
                ptrMethod[entry['t']] = {}
            ptrMethod = ptrMethod[entry['t']]
            
            if entry['family'] not in ptrMethod:
                ptrMethod[entry['family']] = []            
            ptrMethod[entry['family']].append(entry)
            
        else:
            key = line.split(':')[0].replace(" ", "")
            value = line.split(':')[1].replace(" ", "")[:-1]
            if value[-1] == ',':
                value = value[:-1]            
            entry[key] = value     


# Add ktf in the XP.
analyze['ktf'] = {}
for k in kValue:
    analyze['ktf'][k] = {}
    analyze['ktf'][k]['100'] = {}
    ptrMethod = analyze['ktf'][k]['100']
    
    for f in result['ktf'][k]:
        ptrMethod[f] = []
        
        for i in result['ktf'][k][f]:
            if(result['ktf'][k][f][i]['cost'] == 'None'):
                continue
            instance = i.split('.')[0]
            ptrMethod[f].append({'instance' : instance, 'family' : f, 'method' : 'ktf', 'k' : k, 't' : '100', 'cost' : result['ktf'][k][f][i]['cost'], 'cost_repair' : '0', 'percent_lost' : '1'})            
        
firstLine = ['Method', "k", "t"]
for f in families:
    firstLine.append(familyShortName[f])    
table = [firstLine]
tableElement = []

for method in analyze:
    if(method[0] == '1'):
        continue
    for k in analyze[method]:
        for t in analyze[method][k]:
            elt = [method, k, t]
            for f in families:                
                cpt = 0
                sumCost = 0
                sumCostRepair = 0
                sumCover = 0
                
                if f in analyze[method][k][t]:
                    for i in analyze[method][k][t][f]:
                        entry = i                    
                        if entry['cost_repair'] != 'infinity':
                            cpt += 1
                            sumCost += float(entry['cost'])
                            sumCostRepair += float(entry['cost']) + float(entry['cost_repair'])
                            sumCover += float(entry['percent_lost'])
                            
                    if cpt != 0:
                        elt.append("{:.2f}".format(sumCost / cpt) + ' ' + "{:.2f}".format(sumCostRepair / cpt) + ' ' + "{:.10f}".format(sumCover / cpt))
                    else:
                        elt.append("NA")
                else:
                    elt.append("NA")
            tableElement.append(elt)

tableElement.sort(key=lambda x: (x[1], x[0], x[2]))
for l in tableElement:
    table.append(l)
    
display_table(table)

As for the previous table but here we only consider benchmarks for a families and a value k that have been solved by all the method.

In [ ]:
# first, compute for each k the posible benchmark we consider
instanceAvailable = {}
nbMethod = 0

for k in kValue:
    instanceAvailable[k] = {}           
    for inst in instances:            
        instanceAvailable[k][inst.split('.')[0]] = 0

for method in analyze:
    if(method[0] == '1'):
        continue
    
    for k in analyze[method]:        
        for t in analyze[method][k]:
            if k == '1':
                nbMethod += 1
            
            for f in analyze[method][k][t]: 
                for entry in analyze[method][k][t][f]:                      
                    instanceAvailable[k][entry['instance']] += 1                    
                                                            
instanceToPrint = {}
for k in instanceAvailable:
    instanceToPrint[k] = set()
    for i in instanceAvailable[k]:
        if instanceAvailable[k][i] == nbMethod:
            instanceToPrint[k].add(i)

            
firstLine = ['Method', "k", "t"]
for f in families:
    firstLine.append(familyShortName[f])    
table = [firstLine]
tableElement = []

for method in analyze:
    if(method[0] == '1'):
        continue
    for k in analyze[method]:
        for t in analyze[method][k]:
            elt = [method, k, t]
            for f in families:                
                cpt = 0
                sumCost = 0
                sumCostRepair = 0
                sumCover = 0
                
                if f in analyze[method][k][t]:
                    for i in analyze[method][k][t][f]:
                        entry = i                    
                        
                        if entry['instance'] not in instanceToPrint[k]:
                            continue
                        
                        if entry['cost_repair'] != 'infinity':
                            cpt += 1
                            sumCost += float(entry['cost'])
                            sumCostRepair += float(entry['cost']) + float(entry['cost_repair'])
                            sumCover += float(entry['percent_lost'])
                            
                    if cpt != 0:
                        elt.append("{:.2f}".format(sumCost / cpt) + ' ' + "{:.2f}".format(sumCostRepair / cpt) + ' ' + "{:.10f}".format(sumCover / cpt))
                    else:
                        elt.append("NA")
                else:
                    elt.append("NA")
            tableElement.append(elt)

tableElement.sort(key=lambda x: (x[1], x[0], x[2]))
for l in tableElement:
    table.append(l)
    
display_table(table)